In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam

In [2]:
eurusd = pd.read_csv('data/EUR_USD Historical Data.csv')
eurusd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      258 non-null    object 
 1   Price     258 non-null    float64
 2   Open      258 non-null    float64
 3   High      258 non-null    float64
 4   Low       258 non-null    float64
 5   Vol.      0 non-null      float64
 6   Change %  258 non-null    object 
dtypes: float64(5), object(2)
memory usage: 14.2+ KB


In [3]:
eurusd.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')

In [4]:
eurusd = eurusd.drop(columns=['Vol.','Change %'])
eurusd.columns = eurusd.columns.str.strip()
eurusd['Date'] = pd.to_datetime(eurusd['Date'])
eurusd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    258 non-null    datetime64[ns]
 1   Price   258 non-null    float64       
 2   Open    258 non-null    float64       
 3   High    258 non-null    float64       
 4   Low     258 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 10.2 KB


In [5]:
close_data = eurusd.filter(['Price'])
df = close_data.values
train_size = int(np.ceil(len(df) * .7))#slice data
train_size

181

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

train_data = scaled_data[0:int(train_size), :]
# prepare feature and labels
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [7]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128, return_sequences=True, input_shape=(x_train.shape[1], 1))))
model.add(BatchNormalization())
model.add(LSTM(units=64, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

c:\Users\rnmpr\Documents\project n sample\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
history = model.fit(x_train,
                    y_train,
                    epochs=10)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - loss: 0.1672 - mae: 0.3332
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.0559 - mae: 0.1820
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0371 - mae: 0.1501
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0257 - mae: 0.1232
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0261 - mae: 0.1233
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.0252 - mae: 0.1247
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0183 - mae: 0.0990
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.0162 - mae: 0.0985
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0105 - mae: 0.0805
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0133 - mae: 0.0957


In [9]:
model.save('model.keras')

In [10]:
print("\ninitial mae {} | latest mae: {}".format(history.history["mae"][0], history.history["mae"][-1]))
print("initial loss: {} | latest loss: {}".format(history.history["loss"][0], history.history["loss"][-1]))


initial mae 0.27357274293899536 | latest mae: 0.0908128097653389
initial loss: 0.12040668725967407 | latest loss: 0.012494013644754887


In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("model.keras")

# Load new data for predictions
new_data = pd.read_csv("data/EUR_USD Historical Data.csv")
new_data['Date'] = pd.to_datetime(new_data['Date'])
new_data.set_index('Date', inplace=True)

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(new_data[['Price']].values)

# Prepare data for prediction# Sliding window size
lookback = 60
X_pred = [scaled_data[i-lookback:i, 0] for i in range(lookback, len(scaled_data))]
X_pred = np.array(X_pred).reshape(len(X_pred), lookback, 1)

# Predict
try:
    predictions = model.predict(X_pred)
except ValueError as e:
    print("Error during prediction:", e)

predicted_prices = scaler.inverse_transform(predictions)

# Save predictions
prediction_dates = new_data.index[lookback:]
# Ensure matching lengths
min_length = min(len(prediction_dates), len(predicted_prices.flatten()))
prediction_dates = prediction_dates[:min_length]
predicted_prices = predicted_prices[:min_length]

# Create DataFrame
results = pd.DataFrame({"Date": prediction_dates, "Predicted_Price": predicted_prices.round(4).flatten() })
results.to_csv("predicted_price.csv", index=False)

print("Predictions saved to 'predicted_price.csv'")



7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step
Predictions saved to 'predicted_price.csv'
